- SCHERRER Arthur
- CHIRON Yoann
- BOQUAIN Mathis

# Classifieurs - Spotify Best Songs from 2000 to 2023
-> Prédiction du genre des musiques populaires de 2000 à 2023

### Import des différentes librairies

In [ ]:
import pandas as pd
import numpy as np

### Chargement des données

In [ ]:
data = pd.read_csv("Best_Songs_Spotify_2000_2023.csv")

### Affichage

In [ ]:
print(data.head())

### Discrétisation des données

#### Feature

Danceability :
- 0 < x < 25 : peu dansant
- 26 < x < 50 : moyennement dansant
- 51 < x < 75 : dansant
- 76 < x < 100 : très dansant

Energy :

Energy :
- 0 <= x <= 25 : pas énergique
- 26 <= x <= 50 : peu énergique
- 51 <= x <= 75 : énergique
- 76 <= x <= 100 : très énergique

BPM :

dB :

Liveness :

Valence :

Duration :

Valence : 

Duration : 

Acoustisness :

Speechiness : 

Top Genre :